# CELL TOWER COVERAGE

## Problem Description

A telecommunications company needs to build a set of mobile phone towers to provide signal coverage to the inhabitants of a given city. Several potential locations have been identified where the towers could be built. The towers have a fixed range and due to budget restrictions, only a limited number of them can be built. Given these restrictions, the company wants to provide coverage to the largest possible percentage of the population. To simplify the problem, the company has divided the area it wants to cover into a set of regions, each of which has a known population. The objective is then to choose in which of the potential locations the company should build cell phone towers, to provide coverage to as many people as possible. 

### RANDOM DATA GENERATION

In [2]:
import numpy as np

number_of_towers = 11
number_of_regions = 7

tower_coverage = np.random.randint(0, 2, size=(number_of_towers, number_of_regions))
population_region = np.random.randint(1, 100, size=(number_of_regions))
tower_cost = np.random.randint(1, 100, size=(number_of_towers))
budget = np.random.randint(sum(tower_cost) // 2, sum(tower_cost))

### TOWER COVERAGE BY REGION

In [3]:
print(tower_coverage)

[[1 1 0 0 1 1 0]
 [1 1 0 0 1 1 1]
 [0 0 1 1 1 0 1]
 [1 0 0 1 0 1 1]
 [0 0 1 0 1 0 0]
 [0 0 0 1 1 0 0]
 [0 1 1 0 1 0 1]
 [0 0 1 0 0 0 1]
 [1 0 0 1 1 0 0]
 [0 0 1 0 1 1 1]
 [0 1 1 0 0 1 0]]


### POPULATION BY REGION

In [4]:
print(population_region)

[82 53 22 52 11 60  9]


### TOWER COST

In [5]:
print(tower_cost)

[84 63 28 58 41 77 70 34 41  1 65]


### BUDGET

In [6]:
print(budget)

490


# PEQNP TENSOR MODEL

In [7]:
!pip install peqnp
import peqnp as cnf

X_opt = None
optimal = 0
while True:

    cnf.engine(16)

    X = cnf.tensor(dimensions=(number_of_towers, number_of_regions))

    # ensure that at least one tower that covers a region must be selected.
    for i in range(number_of_towers):
        assert sum(X[[i, j]](0, 1) for j in range(number_of_regions)) <= 1

    # ensure that at least one tower that covers a region must be selected.
    for j in range(number_of_regions):
        assert sum(X[[i, j]](0, 1) for i in range(number_of_towers)) <= 1

    # ensure that the total cost of building towers do not exceed the allocated budget.
    for i in range(number_of_towers):
        assert sum(X[[i, j]](0, tower_cost[i]) for j in range(number_of_regions)) <= budget


    # We seek to maximize the total population covered by the towers.
    assert sum(X[[i, j]](0, population_region[j] * tower_coverage[i, j]) for i in range(number_of_towers) for j in range(number_of_regions)) > optimal

    if cnf.satisfy(turbo=True):
        X_opt = np.vectorize(int)(X.binary)
        optimal = sum(X_opt[i][j] * population_region[j] * tower_coverage[i, j] for i in range(number_of_towers) for j in range(number_of_regions))        
        print(optimal)
    else:
        if X_opt is None:
            print('Infeasible...')   
        else:
            cost = 0
            for i in range(number_of_towers):
                cost += sum(X_opt[i][j] * tower_cost[i] for j in range(number_of_regions))
            
            population_covered = sum(X_opt[i][j] * population_region[j] for i in range(number_of_towers) for j in range(number_of_regions))

            print(80 * '-')
            print('COST     vs BUDGET     : {} vs {}'.format(cost, budget))
            print('COVERING vs POPULATION : {} vs {}'.format(population_covered, sum(population_region)))
            print('TOWERS:')
            for i in range(number_of_towers):
                if sum(X_opt[i]) > 0:
                    print('TOWER Nº {}'.format(i + 1))
        break 

215
227
237
267
278
289
--------------------------------------------------------------------------------
COST     vs BUDGET     : 345 vs 490
COVERING vs POPULATION : 289 vs 289
TOWERS:
TOWER Nº 1
TOWER Nº 2
TOWER Nº 3
TOWER Nº 4
TOWER Nº 7
TOWER Nº 9
TOWER Nº 10


#### Copyright © 2021 PEQNP